# 目标

在已有的温度界面把时间显示出来，设计思路：先获取时间内容并用标签显示在指定位置；然后想办法让时间每秒更新一次。 

为了让时间定时更新,我们还需要一个定时器,就像每天叫我们起床的闹钟一样,定时去改变显示的时间.

![](./image/temp_clock.png)

# 知识点

* QDateTime类使用
* Qtimer类使用
* 信号与槽
* 自定义类
* 自定义信号

# 获取时间，并打印

查阅资料我们了解到PyQt的时间相关类有:`Qdate`, `QTime`, `QDateTime`,下面先导入时间类

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication,  QLabel
from PyQt5.QtCore import QDateTime

In [2]:
print(QDateTime.currentDateTime())

PyQt5.QtCore.QDateTime(2020, 6, 27, 22, 24, 36, 373)


```cpp
// C++版本
// qDebug提供了便于打印调试的方法
// 与C++的cout用法类似
// 可以在编译时关闭打印输出，不用修改代码能够直接发布
#include <QDebug>
qDebug() << "Hello world";

// QDateTime::currentDateTime() 函数获得当前的时间
#include <QDateTime>
qDebug() << "Date:" << QDateTime::currentDateTime();
```

↓ 在Qt Creator环境的_应用程序输出_窗格打印
![](image/qtc_console_qdebug.png)

这个打印结果实际上是`currentDateTime`函数返回数据的打印,可以通`toString`把数据转变为字符串

In [3]:
clock = QDateTime.currentDateTime() 
print(type(QDateTime.currentDateTime()))

<class 'PyQt5.QtCore.QDateTime'>


In [4]:
print(QDateTime.currentDateTime().toString())
print(clock.toString())

周六 6月 27 22:24:36 2020
周六 6月 27 22:24:36 2020


上面的日期确实显示出来了,但是格式不是我们想要的,在toString中增加格式

In [5]:
from PyQt5.QtCore import Qt 
print(clock.toString(Qt.ISODate))
print(clock.toString(Qt.DefaultLocaleLongDate))
print(clock.toString("yyyy-MM-dd hh:mm:ss dddd")) #自定义格式显示时间

2020-06-27T22:24:36
2020年6月27日 22:24:36
2020-06-27 22:24:36 星期六


我们再看看`Qdate`和`QTime`的使用.

In [6]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication,  QLabel
from PyQt5.QtCore import QDate, QTime

print(QDate.currentDate().toString())
print(QDate.currentDate().toString(Qt.ISODate))
print(QTime.currentTime().toString())
print(QTime.currentTime().toString(Qt.ISODate))

周六 6月 27 2020
2020-06-27
22:24:36
22:24:36


可以发现,QDateTime = QDate + QTime

# 将时间显示在窗口左下角

先在空白的窗口上显示时间数据进行测试,重要是配置显示位置,字体大小,标签尺寸

In [7]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QDateTime

app = QApplication([])
widgetMainWindow = QWidget()
widgetMainWindow.setWindowTitle("温度") #修改标题内容
widgetMainWindow.resize(250,150) #修改窗口大小
widgetMainWindow.setStyleSheet("background-color: black") #修改窗体背景颜色

labelClock = QLabel()
currentClock = QDateTime.currentDateTime()
labelClock.setText(currentClock.toString("yyyy-MM-dd hh:mm:ss dddd"))
labelClock.setParent(widgetMainWindow)

In [8]:
labelClock.setFont(QFont("Microsoft YaHei",10)) # 字体不需要很大
labelClock.setStyleSheet("background-color: white;color:rgb(255,0,255,255)") # 修改背景为白色,方便确认布局
labelClock.setGeometry(0, 120, 250, 30) # 左下角起始位置为0,120, 长250,高30

In [9]:
widgetMainWindow.show()
app.exec_()

0

```cpp
// C++版本
#include <QString>
// 定义一个QLabel的子类
// 为什么要新定义？
// 功能复杂，却又比较独立，因此进行封装
class QLabelClock: public QLabel
{
private:
    QString now()
    {
        return QDateTime::currentDateTime().toString("yyyy-MM-dd hh:mm:ss dddd");
    }
public:
    QLabelClock(QWidget* parent):
        QLabel(parent)
    {
        this->setText(now());
    }
};

// ... main函数内部
// 在主窗口上增加一个时钟控件
    QLabelClock *labelClock = new QLabelClock(&w);
// 一定要放在窗口的show之前，原因后续再解释
    w.show();
```

# 函数封装

当我们要把温度显示部分增加进来的时候发现,代码写的好长了,看起来也有点费力.不过我们知道的是哪些行是干什么的,例如上面新添加的`labelClock`相关的代码就是时间显示相关的,之前温度显示部分的代码是与温度有关的即便当时设计了3个标签.而`QApplication`, `QWidget`和`app.exec_`是基础平台相关的.所以我们可以把代码分为三个部分,也就是三个函数:

In [10]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QDateTime, QTimer

# 温度显示函数
def tempDisplay(parent): 
    labelTemperatureText = QLabel()
    labelTemperatureText.setText("T-")
    labelTemperatureText.setFont(QFont("Microsoft YaHei",15))
    labelTemperatureText.setStyleSheet("background-color: black;color:rgb(255,0,255,255)")
    labelTemperatureText.setGeometry(0, 50, 50, 50)

    labelTemperatureData = QLabel()
    labelTemperatureData.setText("36.5")
    labelTemperatureData.setFont(QFont("Microsoft YaHei",45))
    labelTemperatureData.setStyleSheet("background-color: black;color:rgb(255,0,255,255)")
    labelTemperatureData.setGeometry(50, 0, 150, 150)

    labelTemperatureSymbol = QLabel()
    labelTemperatureSymbol.setText("\260C")
    labelTemperatureSymbol.setFont(QFont("Microsoft YaHei",15))
    labelTemperatureSymbol.setStyleSheet("background-color: black;color:rgb(255,0,255,255)")
    labelTemperatureSymbol.setGeometry(200, 0, 50, 50)
    
    if parent is not None:
        labelTemperatureSymbol.setParent(parent)
        labelTemperatureData.setParent(parent)
        labelTemperatureText.setParent(parent)

# 时间显示函数
def clockDisplay(parent):
    labelClock = QLabel()
    currentClock = QDateTime.currentDateTime()
    labelClock.setText(currentClock.toString("yyyy-MM-dd hh:mm:ss dddd"))
    labelClock.setFont(QFont("Microsoft YaHei",10))
    labelClock.setStyleSheet("background-color: black; color: white")
    labelClock.setGeometry(0, 120, 250, 30)
    
    if parent is not None:
        labelClock.setParent(parent)
        
# 主函数
def main():
    app = QApplication([])
    
    widgetMainWindow = QWidget()
    widgetMainWindow.setWindowTitle("温度") #修改标题内容
    widgetMainWindow.resize(250,150) #修改窗口大小
    widgetMainWindow.setStyleSheet("background-color: black") #修改窗体背景颜色
    
    tempDisplay(widgetMainWindow)
    clockDisplay(widgetMainWindow)
    
    widgetMainWindow.show()

    app.exec_()

main()


# 让时间自动更新

时间的自动更新需要用到定时器，就是让时间每秒钟更新一次。定时时间到的时候执行一个更新时间的函数即可，实际上定时时间和更新时间是2个不同的事情，那么QT中是如何让这两个事情联系起来的呢？ 很快想到了回调函数，查资料时又发现了信号\槽机制，这里面有什么区别呢？ 我们请教了公司的大牛。

大牛听了我的问题后，喝了一口咖啡，说：“你知道回调函数吧” 

我：“知道，回调函数就是一个通过函数指针调用的函数。如果你把函数的指针（地址）作为参数传递给另一个函数，当这个指针被用来调用其所指向的函数时，我们就说这是回调函数。回调函数不是由该函数的实现方直接调用，而是在特定的事件或条件发生时由另外的一方调用的，用于对该事件或条件进行响应。” 这是我请教大牛之前百度过的

大牛：“嗯，看来你还做过一些研究，那就好说了，回调函数的本质是把调用者和被调用这分开，即解耦。 借用别人的总计'你想让别人的代码执行你的代码，而别人的代码你是不能动的'， 如果同一个事件发生时需要调用多个回调函数，那么就需要维护一个列表， 并且每一个事件都需要维护这样的列表，这种情况下效率就会很低，调用者与被调用者的解耦程度不够” 

大牛：“qt中为了解决这个问题，设计了信号与槽机制， 编程时只需要绑定信号(事件)和槽函数(等价于回调函数)， 当信号函数执行时，qt会找到并执行与其绑定的槽函数， 并且qt允许一个信号绑定多个槽函数。注意这里说的是qt会自动完成匹配，而不需要编程者维护任何列表。 信号与槽机制降低了qt对象的耦合度。” 

大牛：“所以这里有好用的信号与槽机制，为什么不用呢。不过我要提醒的一点是因为qt进行了一层封装，因此执行效率上可能会有些影响，不过以目前处理器的速度来说区别只是几个微妙，可忽略的。” 

我：“所以我把定时器时间到之类的信号和时间刷新函数之间进行绑定就可以了是吗？” 本着有机会就多问点，我说出了自己理解

大牛：“没错，看来你已经理解了信号与槽的关键，再去尝试下就能掌握这个技能了” 

我：“刚才您提到信号与槽机制降低了QT对象的耦合度，那么是不是说这个机制是用在对象之间的？” 

大牛：“没错，你可以通过调用 QObject 对象的 connect 函数来将某个对象的信号与另外一个对象的槽函数相关联，这样当发射者发射信号时，接收者的槽函数将被调用。” 

经过大牛的指点，我们豁然开朗。为了不引入新的问题，暂时不添加温度显示部分内容进行研究。我们需要用到是定时器的信号，因此connect连接的一个对象是定时器，另一对象就是我们的界面上的时钟了，所以这里需要先创建一个时钟窗口类如下：

## 自定义类WinClock

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QDateTime, QTimer

class WinClock():
    def __init__(self,parent): # 这个初始化函数在类的实例化时会被自动调用
        self.labelClock = QLabel()
        self.labelClock.setFont(QFont("Microsoft YaHei",10))
        self.labelClock.setStyleSheet("background-color: black;color:white")
        self.labelClock.setGeometry(0, 120, 250, 30)
        if parent is not None:
            self.labelClock.setParent(parent)

    def slotClockUpdate(self):  # 更新时钟槽函数
        datatime = QDateTime.currentDateTime()
        self.labelClock.setText(datatime.toString("yyyy-MM-dd hh:mm:ss dddd")) # 修改显示内容
    

上面定义了一个QTimer对象，定义了一个`slotClockUpdate`函数，然后使用self.timer.timeout.connect(self.slotClockUpdate)把两者建立连接。  
有规定类名称的首字母大写，例如QWidget，QTimer等都是采用这样的规则，这个规则是为了让代码可读性更好，解释器并不会因为是小写字母而报错或执行错误。
> 规定代码中类名称首字母大写，且使用驼峰命名风格，私类下划线开头

In [2]:
def main():
    app = QApplication([])
    widgetMainWindow = QWidget()
    widgetMainWindow.setWindowTitle("温度") #修改标题内容
    widgetMainWindow.resize(250,150) #修改窗口大小
    widgetMainWindow.setStyleSheet("background-color: black") #修改窗体背景颜色
    
    clockDisplay = WinClock(widgetMainWindow)
    
    timer = QTimer()
    timer.start(1000) # 定时器周期为1000ms
    timer.timeout.connect(clockDisplay.slotClockUpdate) # timer的timeout信号与clock_display的槽函数slot_clock_update进行绑定

    widgetMainWindow.show()

    app.exec_()

main()


上面代码中实现的是定时器与时钟显示的槽函数绑定的方法。为了代码让时钟显示代码更加收敛，我们可以吧定时器的实例化和绑定过程与WinClock的实例化过程进行合并。因为WinClock类中的`__init__`函数在实例化过程中会被自动调用，那么我们同样可以把timer相关代码放到这个函数中。

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QDateTime, QTimer

class WinClock():
    def __init__(self,parent): # 这个初始化函数在类的实例化时会被自动调用
        self.labelClock = QLabel()
        self.labelClock.setFont(QFont("Microsoft YaHei",10))
        self.labelClock.setStyleSheet("background-color: black;color:white")
        self.labelClock.setGeometry(0, 120, 250, 30)
        if parent is not None:
            self.labelClock.setParent(parent)

        self.timer = QTimer() # 实例定时器对象
        self.timer.timeout.connect(self.slotClockUpdate) # 实例对象与当前类中的槽函数进行绑定
        self.timer.start(1000)

    def slotClockUpdate(self):  # 更新时钟槽函数
        datatime = QDateTime.currentDateTime()
        self.labelClock.setText(datatime.toString("yyyy-MM-dd hh:mm:ss dddd")) # 修改显示内容
        
def main():
    app = QApplication([])
    widgetMainWindow = QWidget()
    widgetMainWindow.setWindowTitle("温度") #修改标题内容
    widgetMainWindow.resize(250,150) #修改窗口大小
    widgetMainWindow.setStyleSheet("background-color: black") #修改窗体背景颜色
    
    clockDisplay = WinClock(widgetMainWindow)

    widgetMainWindow.show()

    app.exec_()

main()

```cpp
// C++版本
#include <QTimer>
//在class定义内部增加一个定时器
QTimer *t;
//响应定时器事件的槽函数
void refresh()
{
    this->setText(now());
}

//构造函数内部
//实例化定时器
t = new QTimer(parent);
//连接信号和槽
//定时器每隔若干毫秒，产生一次信号（事件）
//connect设定了信号与响应的槽函数之间的关系
//即每次触发信号，refresh函数被调用一次
connect(t, &QTimer::timeout, this, &QLabelClock::refresh);
t->start(1000);  //设定周期为1000毫秒，即1秒
```

可以看到，时钟不停刷新

# 完成代码

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QDateTime, QTimer

# 温度显示函数
def tempDisplay(parent): 
    labelTemperatureText = QLabel()
    labelTemperatureText.setText("T-")
    labelTemperatureText.setFont(QFont("Microsoft YaHei",15))
    labelTemperatureText.setStyleSheet("background-color: black;color:rgb(255,0,255,255)")
    labelTemperatureText.setGeometry(0, 50, 50, 50)

    labelTemperatureData = QLabel()
    labelTemperatureData.setText("36.5")
    labelTemperatureData.setFont(QFont("Microsoft YaHei",45))
    labelTemperatureData.setStyleSheet("background-color: black;color:rgb(255,0,255,255)")
    labelTemperatureData.setGeometry(50, 0, 150, 150)

    labelTemperatureSymbol = QLabel()
    labelTemperatureSymbol.setText("\260C")
    labelTemperatureSymbol.setFont(QFont("Microsoft YaHei",15))
    labelTemperatureSymbol.setStyleSheet("background-color: black;color:rgb(255,0,255,255)")
    labelTemperatureSymbol.setGeometry(200, 0, 50, 50)
    
    if parent is not None:
        labelTemperatureSymbol.setParent(parent)
        labelTemperatureData.setParent(parent)
        labelTemperatureText.setParent(parent)

class WinClock():
    def __init__(self,parent): 
        self.labelClock = QLabel()
        self.labelClock.setFont(QFont("Microsoft YaHei",10))
        self.labelClock.setStyleSheet("background-color: black;color:white")
        self.labelClock.setGeometry(0, 120, 250, 30)
        if parent is not None:
            self.labelClock.setParent(parent)

        self.timer = QTimer() # 实例定时器对象
        self.timer.timeout.connect(self.slotClockUpdate) # 实例对象与当前类中的槽函数进行绑定
        self.timer.start(1000)

    def slotClockUpdate(self):  # 更新时钟槽函数
        datatime = QDateTime.currentDateTime()
        self.labelClock.setText(datatime.toString("yyyy-MM-dd hh:mm:ss dddd")) # 修改显示内容
    
        
# 主函数
def main():
    app = QApplication([])
    
    widgetMainWindow = QWidget()
    widgetMainWindow.setWindowTitle("温度") #修改标题内容
    widgetMainWindow.resize(250,150) #修改窗口大小
    widgetMainWindow.setStyleSheet("background-color: black") #修改窗体背景颜色
    
    tempDisplay(widgetMainWindow)
    clock = WinClock(widgetMainWindow)
    
    widgetMainWindow.show()

    app.exec_()

main()


# 挑战游戏设计与实现

游戏规则是这样的：
* 窗口会显示时钟，并且时钟需显示毫秒
* 鼠标单击时钟后停止走时，再次单击继续走时
* 挑战的任务是：通过按钮让时钟停在000毫秒, 对手是自己



如和让一个标签支持按键功能呢。QLabel本身并不支持鼠标功能，因此需要继承QLabel重新创建一个类实现这个功能。原理鼠标按下后会给Qt产生一个鼠标按下时间，新建标签类支持按键事件处理方法/函数，函数中实现按键事件检测，如果是鼠标左键释放按键，则说明有鼠标单击发生，此时通过emit方法发出一个信号。在时间显示类中使用新创建的类显示时间，然后增加槽函数与单击事件绑定即可。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtCore import pyqtSignal

class ClickableLabel(QLabel):
    clicked = pyqtSignal() # 创建一个信号
    
    def mousePressEvent(self, QMouseEvent): # 鼠标按下事件处理函数
        if QMouseEvent.button() == Qt.LeftButton: # 确认是鼠标左键
            self.clicked.emit() # 触发信号函数，执行后会触发与其绑定的槽函数的执行

这里是一个很简单的自定义信号的实现，pyqtSignal类创建了一个信号对象clicked，这个对象可通过函数emit发射一个信号。

这信号与定时器中的timeout是一样的。当信号发出qt底层会自动执行与之绑定的槽函数。

In [3]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QDateTime, QTimer, Qt, pyqtSignal

class ClickableLabel(QLabel):
    clicked = pyqtSignal() # 创建一个信号
    
    def mousePressEvent(self, QMouseEvent): # 鼠标按下事件处理函数
        if QMouseEvent.button() == Qt.LeftButton: # 确认是鼠标左键
            self.clicked.emit() # 触发信号函数，执行后会触发与其绑定的槽函数的执行

class GameClock():
    def __init__(self, parent=None): 
        self.labelClock = ClickableLabel()
        self.labelClock.setWindowTitle("ClockGame")
        self.labelClock.setFont(QFont("Microsoft YaHei",10))
        self.labelClock.setStyleSheet("background-color: black;color:white")
        self.labelClock.resize(300, 30)
        self.labelClock.show()
        self.clockUpdateEn = 1;
        if parent is not None:
            self.labelClock.setParent(parent)

        self.timer = QTimer() # 实例定时器对象
        self.timer.timeout.connect(self.slot_clock_update) # 实例对象与当前类中的槽函数进行绑定
        self.timer.start(1) # 需要每毫秒刷新一次
        
        self.labelClock.clicked.connect(self.slot_mouse_clicked) # 绑定标签鼠标单击信号的槽函数

    def slot_clock_update(self):  # 更新时钟槽函数
        if self.clockUpdateEn == 1:
            datatime = QDateTime.currentDateTime()
            self.labelClock.setText(datatime.toString("yyyy-MM-dd hh:mm:ss.zzz")) # 修改显示内容
    
    def slot_mouse_clicked(self): # 鼠标单击事件处理槽函数
        if self.clockUpdateEn != 0:
            self.clockUpdateEn = 0
        else:
            self.clockUpdateEn = 1
        
# 主函数
def main():
    app = QApplication([])

    clock_display = GameClock()

    app.exec_()

main()

# 任务1

* 将温度显示部分封装为类
* 使用信号与槽机制，使用定时器使温度值在36.0-37.9度之间循环变化，每0.5s步进0.1度

# 任务2

* 使用信号与槽机制更新时间，同时更新显示样式：时钟字体颜色：红、黄、蓝循环显示
* 一个信号触发2个槽函数
* 研究LCDnumber控件，并显示时钟
* 扩展任务，实现挑战游戏
效果：
![](./image/LCDNumber_demo.png)

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QLCDNumber
from PyQt5.QtCore import QDateTime, QTimer

class WinClock():
    def __init__(self): 
        self.lcdClock = QLCDNumber()
        self.lcdClock.setWindowTitle("Clock")
        self.lcdClock.setGeometry(200, 200, 500, 150)
        self.lcdClock.setDigitCount(20)
        self.lcdClock.setSegmentStyle(QLCDNumber.Flat)
        self.lcdClock.setStyleSheet("background-color: white;color:white")
        self.lcdClock.show()
        self.style = 0

        self.timer = QTimer() # 实例定时器对象
        self.timer.timeout.connect(self.slotClockUpdate) # 实例对象与当前类中的槽函数进行绑定
        self.timer.timeout.connect(self.slotClockStyle) # 实例对象与当前类中的槽函数进行绑定
        self.timer.start(1000)

    def slotClockUpdate(self):  # 更新时钟槽函数
        datatime = QDateTime.currentDateTime()
        self.lcdClock.display(datatime.toString("yyyy-MM-dd hh:mm:ss")) # 修改显示内容
        
    def slotClockStyle(self):  # 更新显示样式槽函数
        if self.style == 0:
            self.lcdClock.setStyleSheet("background-color: white;color:red")
            self.style = self.style + 1
        elif self.style == 1:
            self.lcdClock.setStyleSheet("background-color: white;color:green")
            self.style = self.style + 1
        elif self.style == 2:
            self.lcdClock.setStyleSheet("background-color: white;color:blue")
            self.style = 0
        else:
            self.style = 0
    
# 主函数
def main():
    app = QApplication([])

    clockDisplay = WinClock()

    app.exec_()

main()
